In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import shutil

import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
TRAIN_DIR = "/content/drive/MyDrive/White Noise project/aataset/clean_train"
TEST_DIR = "/content/drive/MyDrive/White Noise project/aataset/clean_test"

BATCH_SIZE = 2
EPOCHS = 10
DATA_LEN = 2500

In [ ]:
train_images = glob.glob(TRAIN_DIR+"/*")
test_images = glob.glob(TEST_DIR+"/*")

In [ ]:
def down_sample_gen(neuran, input):

    lstm0 = layers.LSTM(neuran, return_sequences=True)(input)
    lstm1 = layers.LSTM(neuran, return_sequences=True)(input)
    attention = layers.Attention()([lstm0, lstm1])

    return attention

In [ ]:
def up_sample_gen(neuran, input, layer):

    c = tf.keras.layers.concatenate([input, layer], axis = 2)
    lstm0 = layers.LSTM(neuran, return_sequences=True)(input)
    lstm1 = layers.LSTM(neuran, return_sequences=True)(input)
    attention = layers.Attention()([lstm0, lstm1])

    return attention

In [ ]:
def build_generator():

    input_layer = tf.keras.Input(shape=( 64, 2))

    ds0 = down_sample_gen(32, input_layer)

    ds1 = down_sample_gen(64, ds0)

    u0 = layers.LSTM(128, return_sequences=True)(ds1)
    u1 = layers.LSTM(128, return_sequences=True)(u0)

    us0 = up_sample_gen(64, u1, ds1)

    us1 = up_sample_gen(32, us0, ds0)

    o = layers.LSTM(16, return_sequences = True)(us1)

    return tf.keras.Model(inputs=input_layer, outputs=us1)

In [ ]:
generator = build_generator()

In [ ]:
generator.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 64, 2)]              0         []                            
                                                                                                  
 lstm_13 (LSTM)              (None, 64, 32)               4480      ['input_3[0][0]']             
                                                                                                  
 lstm_14 (LSTM)              (None, 64, 32)               4480      ['input_3[0][0]']             
                                                                                                  
 attention_5 (Attention)     (None, 64, 32)               0         ['lstm_13[0][0]',             
                                                                     'lstm_14[0][0]']       

In [ ]:
def down_sample_disc(filters, input):

    cnn0 = layers.Conv2D(filters = filters, kernel_size = (3, 3), activation = tf.keras.activations.relu)(input)
    cnn1 = layers.Conv2D(filters = filters, kernel_size = (3, 3), activation = tf.keras.activations.relu)(cnn0)
    mp   = layers.MaxPool2D()(cnn1)

    return mp

In [ ]:
def build_discriminator():

    input_layer = tf.keras.Input(shape=( 64, 64, 2))

    ds0 = down_sample_disc(16, input_layer)

    ds1 = down_sample_disc(32, ds0)

    ds2 = down_sample_disc(64, ds1)

    flatten = layers.Flatten()(ds2)

    dnn = layers.Dense(32, activation='relu')(flatten)
    dnn = layers.Dense(16, activation='relu')(dnn)
    dnn = layers.Dense(1, activation='sigmoid')(dnn)

    return tf.keras.Model(inputs=input_layer, outputs=dnn)

In [ ]:
discriminator = build_discriminator()

In [ ]:
discriminator.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 64, 64, 2)]       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 62, 62, 16)        304       
                                                                 
 conv2d_3 (Conv2D)           (None, 60, 60, 16)        2320      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 28, 28, 32)        4640      
                                                                 
 conv2d_5 (Conv2D)           (None, 26, 26, 32)        9248      
                                                           

In [ ]:
def discriminator_loss(real_output, fake_output):

    real_loss = tf.keras.losses.BinaryCrossentropy(tf.ones_like(real_output), real_output)
    fake_loss = tf.keras.losses.BinaryCrossentropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
def generator_loss(fake_output):
    return tf.keras.losses.BinaryCrossentropy(tf.ones_like(fake_output), fake_output)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
checkpoint_dir = '/content/drive/MyDrive/White Noise project/code/runs/first'
checkpoint_prefix = checkpoint_dir + "/ckpt"
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)


In [ ]:
train_datagen = datagen()
for epoch in EPOCHS:
  for step in range(0, DATA_LEN // BATCH_SIZE, BATCH_SIZE):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(train_datagen[0])

      real_output = discriminator(train_datagen[1])
      fake_output = discriminator(generated_images)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))